In [14]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import types
from pyspark.sql import functions as F

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

In [9]:
spark.version

'3.3.2'

In [10]:
schema = types.StructType([
    types.StructField('dispatching_base_num', types.StringType(), True),
    types.StructField('pickup_datetime', types.TimestampType(), True),
    types.StructField('dropoff_datetime', types.TimestampType(), True),
    types.StructField('PULocationID', types.IntegerType(), True),
    types.StructField('DOLocationID', types.IntegerType(), True),
    types.StructField('SR_Flag', types.StringType(), True),
    types.StructField('Affiliated_base_number', types.StringType(), True)
])

In [12]:
df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv('data/ny_taxi_data/FHVHV_NY_Taxi_data_2019-2021/fhvhv_tripdata_2021-06.csv')

df = df.repartition(12)

df.write.parquet('data/pq/fhvhv/2021/06/')

In [15]:
df.withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .filter("pickup_date = '2021-06-15'") \
    .count()

452470

In [34]:
df.withColumn('trip_duration_in_hrs', \
              (F.col("dropoff_datetime").cast("long") - F.col('pickup_datetime').cast("long"))/3600) \
.select(F.col('trip_duration_in_hrs')).sort(F.col('trip_duration_in_hrs').desc()).limit(1).show()

+--------------------+
|trip_duration_in_hrs|
+--------------------+
|    66.8788888888889|
+--------------------+



In [37]:
spark.read \
    .option("header", "true") \
    .csv('taxi+_zone_lookup.csv').createOrReplaceTempView("taxi_lookup_v")

In [40]:
spark.sql("""select * from taxi_lookup_v""").show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [47]:
df.createOrReplaceTempView("fhvhv_2021_06_v")

In [49]:
spark.sql("""select b.Zone as zone, count(1) as count from
fhvhv_2021_06_v a,
taxi_lookup_v b
where a.PULocationID = b.LocationID
group by zone""").sort(F.col('count').desc()).limit(1).show()

+-------------------+------+
|               zone| count|
+-------------------+------+
|Crown Heights North|231279|
+-------------------+------+

